# Imports and Constants

In [ ]:
import numpy as np
import sklearn
import random

from loadData import get_maps,get_masks
from masks import apply_mask_to_maps, get_part_of_maps
from machineLearning import Decoder

SEED = 0
random.seed(SEED)
n_subjects = 2
classes = ['Up','Down','Right','Left']
nb_runs = 12
length = nb_runs*len(classes)

# Loading data

In [2]:
# create labels
class_labels = [None]*(2*length)

for i in range(length*2) :
    class_labels[i]=classes[(i // nb_runs) % len(classes)]
labels = dict()
labels["vis"] = np.array(class_labels[:length])
labels["aud"]= np.array(class_labels[length:])

In [3]:
t_maps,beta_maps = get_maps(n_subjects)
masks,masks_full = get_masks(n_subjects,plot=False)
maps = apply_mask_to_maps(t_maps,masks)
maps_masked = dict()
maps_masked["vis"] = get_part_of_maps(maps,0,length) # maps acquired for the vision experiment
maps_masked["aud"] = get_part_of_maps(maps,length,2*length) # maps acquired for the audition experiment

# Motion decoding

In [10]:
classifier = sklearn.svm.SVC(C=1,random_state=SEED) #default parameters, there is clearly improvements to be made here
decoder = Decoder(n_perm=1000,model=classifier,n_classes=len(classes),n_splits=nb_runs,seed=SEED)
cv_scores = dict()
p_values = dict()

#### Visual motion using V5

In [11]:
tasks = ["vis"]
regions = ["V5_R","V5_L"]
cv_sc, p_val = decoder.classify_tasks_regions(maps_masked,labels,tasks,regions)
cv_scores.update(cv_sc)
p_values.update(p_val)

#### Auditory motion using PT

In [12]:
tasks = ["aud"]
regions = ["PT_R","PT_L"]
cv_sc, p_val = decoder.classify_tasks_regions(maps_masked,labels,tasks,regions)
cv_scores.update(cv_sc)
p_values.update(p_val)

#### Using V5 to predict auditory direction

In [13]:
tasks = ["aud"]
regions = ["V5_R","V5_L"]
cv_sc, p_val = decoder.classify_tasks_regions(maps_masked,labels,tasks,regions)
cv_scores.update(cv_sc)
p_values.update(p_val)

### Cross modal decoding using V5

In [ ]:
# TRAINING ON VISION, TESTING ON AUDITION, and then reverse
tasks=["vis","aud"]
regions=["V5_R","V5_L"]
scores_cross_mod= decoder.cross_modal_decoding(maps_masked,labels,tasks,regions)